In [1]:
"""Extract gene sequence, translate in alternate reading frames to get non-gene sequence"""

'Extract gene sequence, translate in alternate reading frames to get non-gene sequence'

In [2]:
!pip install biopython

     |████████████████████████████████| 2.3MB 7.2MB/s 


In [0]:
import os
import re
import gzip
import pickle
from Bio import SeqIO
from Bio.Data import CodonTable

In [4]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def check_known_protein(protein_description):
    """filter out hypothetical and putative proteins"""
    is_hypothetical = ('hypoth' in protein_description or 'hetical' in protein_description) # deals with most common misspellings of "hypothetical"
    is_putative = 'putative' in protein_description
    is_known = (not is_hypothetical) and (not is_putative)
    return is_known

def get_CDS_locations(genome_path):
    """exctracts gene locations and seqs from known protein coding genes, only non-hypothetical"""
    gene_loc_list = []
    gene_seq_list = []
    with gzip.open(genome_path, 'rt') as handle: # open file
        for record in SeqIO.parse(handle, "genbank"): # parse each record within file
            feature_list = record.features
            CDS_list = [x for x in feature_list if x.type=='CDS'] # get all CDS features in record
            for CDS in CDS_list:
                try:
                    protein_description = CDS.qualifiers['product'][0]
                    if check_known_protein(protein_description): # only use non-hypothetical proteins
                        gene_loc = CDS.location
                        gene_loc_list.append(gene_loc)

                        gene_seq = CDS.extract(record.seq) # extract locations of CDS
                        gene_seq_list.append(gene_seq)
                except:
                    pass

    return gene_loc_list, gene_seq_list

In [6]:
# ECOLI
# load fasta sequence
d = "drive/My Drive/Colab Notebooks/smaug/data/ecoli_MG1655"
genome_path = os.path.join(d, "GCF_000005845.2_ASM584v2_genomic.fna.gz")

with gzip.open(genome_path, 'rt') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        seq = record.seq

# get gene locations
gbff_path = os.path.join(d, "GCF_000005845.2_ASM584v2_genomic.gbff.gz")
gene_loc_list, gene_seq_list = get_CDS_locations(gbff_path)

# total genes and example loc
print(len(gene_loc_list))
print(gene_loc_list[300])

d = "drive/My Drive/Colab Notebooks/smaug/data"
gene_ORF_path = os.path.join(d, "ecoli_MG1655_geneORFs.pkl")
fake_ORF_path = os.path.join(d, "ecoli_MG1655_fakeORFs.pkl")

3532
[381350:381716](+)


In [11]:
# Staphylococcus aureus
# load fasta sequence
d = "drive/My Drive/Colab Notebooks/smaug/data/refseq_singlestrain/bacteria/Staphylococcus/aureus/JH9"
genome_path = os.path.join(d, "GCF_000016805.1_ASM1680v1_genomic.fna.gz")

with gzip.open(genome_path, 'rt') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        seq = record.seq

# get gene locations
gbff_path = os.path.join(d, "GCF_000016805.1_ASM1680v1_genomic.gbff.gz")
gene_loc_list, gene_seq_list = get_CDS_locations(gbff_path)

# total genes and example loc
print(len(gene_loc_list))
print(gene_loc_list[300])

d = "drive/My Drive/Colab Notebooks/smaug/data"
gene_ORF_path = os.path.join(d, "Staphylococcus_geneORFs.pkl")
fake_ORF_path = os.path.join(d, "Staphylococcus_fakeORFs.pkl")

2351
[397392:397566](+)


In [8]:
# format genes correctly
gene_aa = [x.translate(table=11, to_stop=False) for x in gene_seq_list]
gene_aa_filtered = [x[1:-1][::-1] for x in gene_aa if (x[-1] == "*")] # cut off start and stop codons, and reverse (strand should be 3' to 5' for model)

/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2748: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [9]:
# get subORFs in all alternate reading frames
arf_1 = [x[1:].translate(table=11, to_stop=False) for x in gene_seq_list]
arf_2 = [x[2:].translate(table=11, to_stop=False) for x in gene_seq_list]
arf_r0 = [x.reverse_complement()[0:].translate(table=11, to_stop=False) for x in gene_seq_list]
arf_r1 = [x.reverse_complement()[1:].translate(table=11, to_stop=False) for x in gene_seq_list]
arf_r2 = [x.reverse_complement()[2:].translate(table=11, to_stop=False) for x in gene_seq_list]

/usr/local/lib/python3.6/dist-packages/Bio/Seq.py:2748: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


In [0]:
def extract_fake_ORFs(seq_aa):
    stop_positions = [x.start() for x in re.finditer("\\*", str(seq_aa))]
    try:
        fake_ORFs = [str(seq_aa[stop_positions[i-1]+1 : x])[::-1] for i, x in enumerate(stop_positions)] # cut off stop codons, and reverse (strand should be 3' to 5' for model)
        fake_ORFs = [x for x in fake_ORFs if len(x) > 40] # enforce minimum ORF length
    except:
        return []
    return fake_ORFs

fake_aa = []
# for i, arf in enumerate(arf_1):
#     print(i)
#     if i == 10: 
#         break
arfake_1 = [extract_fake_ORFs(x) for x in arf_1]
arfake_2 = [extract_fake_ORFs(x) for x in arf_2]
arfake_r0 = [extract_fake_ORFs(x) for x in arf_r0]
arfake_r1 = [extract_fake_ORFs(x) for x in arf_r1]
arfake_r2 = [extract_fake_ORFs(x) for x in arf_r2]

for x in arfake_1:
    fake_aa.extend(x)
for x in arfake_2:
    fake_aa.extend(x)
for x in arfake_r0:
    fake_aa.extend(x)
for x in arfake_r1:
    fake_aa.extend(x)
for x in arfake_r2:
    fake_aa.extend(x)

In [0]:
# save
with open(gene_ORF_path, 'wb') as f:
    pickle.dump(gene_aa_filtered, f)
with open(fake_ORF_path, 'wb') as f:
    pickle.dump(fake_aa, f)